Global setup


In [122]:
n=4
k=2
q=41
eta=2
R.<x> = ZZ[]

Bob's setup

In [123]:
Ax = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
Ax  #public

[ 40*x^3 + 35*x^2 + 6*x + 39  21*x^3 + 4*x^2 + 26*x + 39]
[22*x^3 + 12*x^2 + 30*x + 26     x^3 + 21*x^2 + 2*x + 14]

In [124]:
def binomial_centered():
    a1=randrange(0,2)
    a2=randrange(0,2)
    b1=randrange(0,2)
    b2=randrange(0,2)
#    return a1+a2-b1-b2
    return a1-b1
    return 0

In [125]:
sx = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
sx  #private

[   -x^2 - 1]
[x^2 + x - 1]

In [126]:
ex = matrix([[binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()], 
            [binomial_centered()*x^3+binomial_centered()*x^2+binomial_centered()*x+binomial_centered()]])
ex #private

[         -x^3]
[x^3 + x^2 + x]

In [127]:
tx = matrix(((Ax*sx + ex) % (x^n+1) % q))
tx  #public

[-38*x^3 - 13*x^2 + 26*x + 14]
[ -29*x^3 + 40*x^2 + 4*x + 32]

**Matrices for lattice attack**


In [128]:
def negacirculant(v):
    Nshift = matrix([[0, 0, 0, -1],             #Negacyclic shift matrix
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0]])
    return sum([v[i]*Nshift^i for i in range(0, len(v))])

In [129]:
Ax

[ 40*x^3 + 35*x^2 + 6*x + 39  21*x^3 + 4*x^2 + 26*x + 39]
[22*x^3 + 12*x^2 + 30*x + 26     x^3 + 21*x^2 + 2*x + 14]

In [130]:
A = block_matrix([[negacirculant(Ax[0,0].padded_list(4)), negacirculant(Ax[0,1].padded_list(4))], 
                  [negacirculant(Ax[1,0].padded_list(4)), negacirculant(Ax[1,1].padded_list(4))]])
A

[ 39 -40 -35  -6| 39 -21  -4 -26]
[  6  39 -40 -35| 26  39 -21  -4]
[ 35   6  39 -40|  4  26  39 -21]
[ 40  35   6  39| 21   4  26  39]
[---------------+---------------]
[ 26 -22 -12 -30| 14  -1 -21  -2]
[ 30  26 -22 -12|  2  14  -1 -21]
[ 12  30  26 -22| 21   2  14  -1]
[ 22  12  30  26|  1  21   2  14]

In [131]:
tx

[-38*x^3 - 13*x^2 + 26*x + 14]
[ -29*x^3 + 40*x^2 + 4*x + 32]

In [132]:
t = block_matrix([[column_matrix(1, 4, tx[0,0].padded_list(4))],
                [column_matrix(1, 4, tx[1,0].padded_list(4))]])
t

[ 14]
[ 26]
[-13]
[-38]
[---]
[ 32]
[  4]
[ 40]
[-29]

In [133]:
I8 = identity_matrix(8)
I8

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]

**Idea for lattice attack**


In [153]:
s = block_matrix([[column_matrix(1, 4, sx[0,0].padded_list(4))],
                [column_matrix(1, 4, sx[1,0].padded_list(4))]])
s

[-1]
[ 0]
[-1]
[ 0]
[--]
[-1]
[ 1]
[ 1]
[ 0]

In [78]:
e = block_matrix([[column_matrix(1, 4, ex[0,0].padded_list(4))],
                [column_matrix(1, 4, ex[1,0].padded_list(4))]])
e

[ 0]
[-1]
[ 0]
[ 0]
[--]
[ 1]
[-1]
[ 0]
[-1]

In [164]:
block_matrix([[s], [e], [matrix([1])], [matrix([[0],[0],[0],[0],[0],[0],[0],[0]])]])

[-1]
[ 0]
[-1]
[ 0]
[-1]
[ 1]
[ 1]
[ 0]
[--]
[ 0]
[-1]
[ 0]
[ 0]
[ 1]
[-1]
[ 0]
[-1]
[--]
[ 1]
[--]
[ 0]
[ 0]
[ 0]
[ 0]
[ 0]
[ 0]
[ 0]
[ 0]

In [89]:
M*block_matrix([[s], [e], [matrix([1])], [matrix([[0],[2],[2],[2],[0],[0],[0],[0]])]])

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [162]:
M.right_kernel_matrix(basis='LLL').transpose()

[-1 -1  1  1 -2  2 -1  0  2  3 -3  0  1 -1 -1 -1 -2]
[ 0  0 -2 -1  1  0 -2 -2  2 -2 -1 -1  0 -1  2  0  1]
[-1  0  0 -1  1  1  1  2  1  0  0  1 -3  0 -2 -1 -1]
[ 0 -1  4  3 -2  2 -1 -3  5 -1  1  0  2  1  0  0  4]
[-1  1 -2  1 -1 -2  3  0 -2 -3  1  1  0  1  1 -2  2]
[ 1  0  0  4  0  0  1  0  0  0  0  0  2  1 -2  0  1]
[ 1  1  1  0 -1  0  0 -1  0 -1  2 -1  5  0  2  3  2]
[ 0  1 -1 -1  0  3  2  3 -4  2  0  2 -1  0  0 -2  0]
[ 0 -3  1  4 -1  1 -1  5 -1  3 -3  4 -1  0  3  0  3]
[ 0 -4 -2 -2  0  2  6  3  1  8  0 -1 -3  4 -2 -5 -2]
[ 0  1  1 -1  3 -1  0 -2  1  1  0  3  2 -5  2  3  0]
[-1 -4  2  0  6 -2  1  2  2  0 -1 -3  0  5 -4 -3 -1]
[ 0 -4  1  4  1 -1  0 -2 -4 -4  0  0 -2 -2 -1  5  1]
[ 1  1  1  1  0  0 -5 -1  1 -3  1  5 -1  3 -1  2 -1]
[ 1 -4  2  1 -2  1  1  0  0 -1 -1 -1 -4  2  1 -2 -4]
[ 1  2 -1 -1 -1  3  2  2 -1  2 -6 -4 -5  3  1 -1  1]
[ 1  1 -3 -3  2 -3  1  0 -3  0 -1  0 -2 -2 -1 -3 -1]
[ 2  1 -2 -3  5  2 -1  1  0 -1  1  0 -3 -1 -1  0  1]
[ 0  0  5 -4  0  2  0  1  2  2  2  1 -2 -1 -2 

**Eve's attack**

Eve knows $A$ so she can set up $M$:

In [144]:
matrix.options.max_rows(49) 

In [146]:
M = block_matrix([[A, I8, -t, q*I8]])
M

[ 39 -40 -35  -6  39 -21  -4 -26|  1   0   0   0   0   0   0   0|-14| 41   0   0   0   0   0   0   0]
[  6  39 -40 -35  26  39 -21  -4|  0   1   0   0   0   0   0   0|-26|  0  41   0   0   0   0   0   0]
[ 35   6  39 -40   4  26  39 -21|  0   0   1   0   0   0   0   0| 13|  0   0  41   0   0   0   0   0]
[ 40  35   6  39  21   4  26  39|  0   0   0   1   0   0   0   0| 38|  0   0   0  41   0   0   0   0]
[ 26 -22 -12 -30  14  -1 -21  -2|  0   0   0   0   1   0   0   0|-32|  0   0   0   0  41   0   0   0]
[ 30  26 -22 -12   2  14  -1 -21|  0   0   0   0   0   1   0   0| -4|  0   0   0   0   0  41   0   0]
[ 12  30  26 -22  21   2  14  -1|  0   0   0   0   0   0   1   0|-40|  0   0   0   0   0   0  41   0]
[ 22  12  30  26   1  21   2  14|  0   0   0   0   0   0   0   1| 29|  0   0   0   0   0   0   0  41]

In [170]:
H, U = M.transpose().hermite_form(transformation = true)
H.transpose()

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [171]:
U.transpose()

[                            0                             0     -132225184516506207077038        8680821321513935898712                             0     -799562733273062652808717     -106798905119953936258664     -188940993756648897595214                     315205835     1556287353867141913140234      132225184516506207077038       -8680821321513935898712      399801527708610281603617      799562733273062652808717      106798905119953936258664      188940993756648897595214     -601862130450538903469784                             0                             0     5421232565176754490158558     -355913674182071371847192                             0    32782072064195568765157397     4378755109918111386605224     7746580744022604801403774]
[                            0                             0     -296338138860245794926672       19455132118757971588646                             0    -1791950097453425839527714     -239353712314981138927791     -423447489591221881426814        

In [150]:
N = U.transpose()[:,8:25]
N

[                    315205835     1556287353867141913140234      132225184516506207077038       -8680821321513935898712      399801527708610281603617      799562733273062652808717      106798905119953936258664      188940993756648897595214     -601862130450538903469784                             0                             0     5421232565176754490158558     -355913674182071371847192                             0    32782072064195568765157397     4378755109918111386605224     7746580744022604801403774]
[                    706427530     3487893018739964029724035      296338138860245794926672      -19455132118757971588646      896020232967515087294775     1791950097453425839527714      239353712314981138927791      423447489591221881426814    -1348870899597121940166702                             0                             0    12149863693270077591993552     -797660416869076835134486                             0    73469953995590459420636274     9813502204914226696039431    1736

In [161]:
N.transpose().LLL().transpose()

[ 1  2 -1 -1  2  0 -1  1  3  1  0  3 -2 -1  1  1 -2]
[ 0  0  0  2 -1 -2  0  2  0  1 -1  1  0  0 -2  0  1]
[ 1  1  0  0 -1  2  1 -1 -2  0  1  0  0  4  2  0 -2]
[ 0  2 -1 -4  2 -3  0  1  2 -1  0 -1  0  2  0 -4  2]
[ 1 -2  1  2  1  0 -1 -3 -3 -1  1 -1  1 -1 -1  2  3]
[-1  0  0  0  0  0  1 -1  0 -1  0  0  4  1  2 -2 -3]
[-1  0  1 -1  1 -1  0  0  0  0 -1 -2  0 -2 -2 -4  2]
[ 0  3  1  1  0  3 -4 -2 -1  0  2  0  0 -2  0  4 -3]
[ 0  1 -3 -1  1  5  3  1 -2  0  4  3  0 -1 -3  2  2]
[ 0  2 -4  2  0  3  2 -6  5 -4 -1  0 -6 -2  2 -2  2]
[ 0 -1  1 -1 -3 -2  0  0  3  5  3  0  3 -1 -2 -4 -2]
[ 1 -2 -4 -2 -6  2  5 -1 -2 -5 -3  1  2 -3  4  1 -1]
[ 0 -1 -4 -1 -1 -2 -5  0 -2  2  0  0  1 -2  1 -1  1]
[-1  0  1 -1  0 -1  1  5 -2 -3  5 -1  2 -4  1  1 -3]
[-1  1 -4 -2  2  0  0 -1 -1 -2 -1  1 -5 -1 -1  0  0]
[-1  3  2  1  1  2 -1 -2  0 -3 -4  6  0  2 -1 -1 -2]
[-1 -3  1  3 -2  0  1 -1  0  2  0  1  0 -2  1  4  2]
[-2  2  1  2 -5  1  1  1 -2  1  0 -1  3  4  1 -1 -4]
[ 0  2  0 -5  0  1  1  0  1  1  1 -2 -4  3  2 

**Eve's attack showing calculation of transformation matrix**


In [175]:
M1 = block_matrix([[A, I8, -t, q*I8], [identity_matrix(25)]], subdivide=False)
M1

[ 39 -40 -35  -6  39 -21  -4 -26   1   0   0   0   0   0   0   0 -14  41   0   0   0   0   0   0   0]
[  6  39 -40 -35  26  39 -21  -4   0   1   0   0   0   0   0   0 -26   0  41   0   0   0   0   0   0]
[ 35   6  39 -40   4  26  39 -21   0   0   1   0   0   0   0   0  13   0   0  41   0   0   0   0   0]
[ 40  35   6  39  21   4  26  39   0   0   0   1   0   0   0   0  38   0   0   0  41   0   0   0   0]
[ 26 -22 -12 -30  14  -1 -21  -2   0   0   0   0   1   0   0   0 -32   0   0   0   0  41   0   0   0]
[ 30  26 -22 -12   2  14  -1 -21   0   0   0   0   0   1   0   0  -4   0   0   0   0   0  41   0   0]
[ 12  30  26 -22  21   2  14  -1   0   0   0   0   0   0   1   0 -40   0   0   0   0   0   0  41   0]
[ 22  12  30  26   1  21   2  14   0   0   0   0   0   0   0   1  29   0   0   0   0   0   0   0  41]
[  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  

In [176]:
H1 = M1.transpose().hermite_form()
H1.transpose()

[  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0  

In [178]:
N1 = H1.transpose()[:,8:25]
N1

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0]
[  0   0   0   0   0

In [179]:
N1.transpose().LLL().transpose()

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[-1  1 -1  2 -3 -1 -1  0  3  3 -1 -1  2  0  2  2  1]
[ 0 -2  0  0 -1  1  0 -1  0  0  2  2 -1 -1 -1 -1  0]
[-1  2  0  1  0  0  1 -1 -3 -2  1  0  2  1  1 -1 -4]
[ 0  0 -1  2  1  0  0  0 -2  2  3 -4 -2  0 -4  2 -2]
[-1 -1  1 -2  1  0 -1  1 -3 -3  1  2 -3  1 -2  1  1]
[ 1  2  0  0  0 -1  1  2 -2  0  0  0  3  0 -1  0 -1]
[ 1 -2  1  0  2  2  0 -4 -1  0  1 -1 -2 -1 -2  1  2]
[ 0  0  1  3  0 -1 -4 -3  1 -1  1  1  3  2  0  0  2]
[ 0 -3 -3  1 -3 -1  3 -1  0 -2 -2 -1 -2  4 -3  1  1]
[ 0  2 -4  2  0  3  2  0 -2  5 -6  2 -2 -1  2  0  2]
[ 0 -2  1 -1  0 -3  0  2  2  3 -2 -1  2  3  0 